In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install miditoolkit miditok pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.4 MB/s eta 0:00:00:00:01


In [5]:
import os
import json
import miditoolkit
from miditok import REMI  # MIDI Tokenizer
import pandas as pd

# Load emotion labels
label_df = pd.read_csv("/kaggle/input/emopia/label.csv")

# Map 4Q labels to emotion text (ensure you have defined emotion_mapping)
emotion_mapping = {1: "Q1", 2: "Q2", 3: "Q3", 4: "Q4"}
label_df["emotion_text"] = label_df["4Q"].map(emotion_mapping)

# Rename 'ID' to match MIDI filenames
label_df.rename(columns={"ID": "filename"}, inplace=True)
label_df["filename"] = label_df["filename"].apply(lambda x: x + ".mid")

# Initialize MIDI tokenizer
tokenizer = REMI()

# MIDI directory
midi_dir = "/kaggle/input/emopia/midis/midis"
output_file = "/kaggle/working/tokenized_midi_data.json"  # <--- Define output_file here

def midi_to_tokens(midi_path):
    """ Convert MIDI file to tokenized representation """
    midi_obj = miditoolkit.MidiFile(midi_path)
    tokens = tokenizer(midi_obj)
    # Convert TokSequence to list of strings
    return [str(token) for token in tokens]

# Process and save tokenized data
tokenized_data = []
for _, row in label_df.iterrows():
    midi_path = os.path.join(midi_dir, row["filename"])
    if os.path.exists(midi_path):
        tokens = midi_to_tokens(midi_path)
        #print(f"Processing: {row['filename']}")  # Debug print
        tokenized_data.append({
            "filename": row["filename"],
            "emotion_text": row["emotion_text"],
            "tokens": tokens
        })
    else:
        print(f"File not found: {midi_path}")

# Save tokenized data as JSON
try:
    with open(output_file, "w") as f:
        json.dump(tokenized_data, f, indent=4)
    print(f"Tokenized MIDI data saved to {output_file}")
except Exception as e:
    print(f"Error saving JSON: {str(e)}")


<ipython-input-5-7fc504f3c793>:28: UserWarning: You are using a depreciated `miditoolkit.MidiFile` object. MidiTokis now (>v3.0.0) using symusic.Score as MIDI backend. Your file willbe converted on the fly, however please consider using symusic.
  tokens = tokenizer(midi_obj)


File not found: /kaggle/input/emopia/midis/midis/Q1_9v2WSpn4FCw_5.mid
File not found: /kaggle/input/emopia/midis/midis/Q1_9v2WSpn4FCw_6.mid
File not found: /kaggle/input/emopia/midis/midis/Q1_JT1XJnVmABo_2.mid
File not found: /kaggle/input/emopia/midis/midis/Q1_ldCQ6N9G6Mk_2.mid
File not found: /kaggle/input/emopia/midis/midis/Q2_9v2WSpn4FCw_1.mid
File not found: /kaggle/input/emopia/midis/midis/Q2_9v2WSpn4FCw_2.mid
File not found: /kaggle/input/emopia/midis/midis/Q4_JT1XJnVmABo_1.mid
Tokenized MIDI data saved to /kaggle/working/tokenized_midi_data.json
